# Guía estructurada de la Tarea 1

Este notebook resume los hitos principales del enunciado para mantener un plan de trabajo ordenado. Avanza sección por sección y completa cada hito conforme desarrolles tu solución del proyecto.


In [ ]:
# Configuración base para reproducibilidad
import os
import random
from pathlib import Path

import numpy as np

GLOBAL_SEED = 42
DATASET_ID = "010"

random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)


## 1. E–T–P y framing

- Describe la Experiencia (E), la Tarea (T) y el Performance/criterio (P).
- Define el framing principal como clasificación binaria de `purchase` con salida probabilística.
- Justifica si usarás tasas agregadas y la decisión de negocio basada en umbrales.



## 2. Métricas y pérdida

- Usa log-loss (entropía cruzada) como pérdida principal.
- Reporta AUC y Brier score.
- Explica por qué MSE no es adecuado como objetivo principal y relaciónalo con máxima verosimilitud.



## 3. Diseño de validación y control de capacidad

- Define particiones train/valid/test (70/15/15) o k-fold para escoger hiperparámetros.
- Reentrena con los mejores hiperparámetros antes del test.
- Controla capacidad con regularización L2 (parámetro C) y grafica curvas train/valid vs. complejidad o learning curves.
- Explica el trade-off sesgo–varianza.



## 4. Preprocesamiento

- Explora el dataset para identificar duplicados, variables irrelevantes o leakage.
- Define codificación de variables categóricas, escalamiento y manejo de outliers o nulos según corresponda.



## 5. Modelado predictivo

- Entrena al menos dos modelos de clasificación (ej. regresión logística, árbol, random forest, XGBoost).
- Compara su desempeño inicial.



## 6. Evaluación

- Reporta métricas de clasificación: accuracy, precision, recall, F1-score y AUC-ROC.



## 7. Discusión de resultados

- Destaca variables relevantes para los modelos.
- Justifica columnas excluidas (especialmente leaks).
- Propón insights accionables para la empresa.



## 8. Política operativa y sensibilidad

- Formula una regla clara: contactar/ofrecer si la probabilidad estimada supera el umbral t.
- Analiza sensibilidad (ej. utilidad esperada por umbral) y discute implicancias.



## 9. Riesgos y mitigación

- Identifica al menos tres riesgos: leakage, sesgo de muestreo, shift temporal/segmento.
- Propón mitigaciones (auditoría de variables, validación por segmento o fuera de tiempo, calibration, A/B).



## 10. Resultados y conclusiones

- Resume hallazgos clave: saturación, desempeño en test, umbral recomendado.
- Indica cómo debe operar la empresa con el modelo final.



## Extras opcionales

- Mostrar efectos de usar `leak_after_offer` para evidenciar data leakage.
- Calcular métricas de negocio (ej. expected profit) usando `unit_margin_if_buy`.
- Comparar `discount` numérico vs. `discount_bucket`.



## Formato y reproducibilidad

- Declara semilla global y DATASET_ID en la primera celda.
- Fija semillas para NumPy/sklearn y repórtalas.
- Reporta versiones de librerías y hash SHA-256 del CSV.
- Trabaja sólo con el dataset entregado, sin regenerar datos.
- Entrega resultados específicos de tu dataset.



In [ ]:
# Punto 1: E–T–P y framing del proyecto
import textwrap
import pandas as pd

dataset_path = Path(f"T1_{DATASET_ID}_individual.csv")
df = pd.read_csv(dataset_path)

FEATURE_COLUMNS = [
    'segment',
    'discount',
    'age',
    'tenure_months',
    'income_index',
    'web_visits_30d',
    'unit_margin_if_buy',
    'discount_bucket',
]
TARGET_COLUMN = 'purchase'
CATEGORICAL_FEATURES = ['segment', 'discount_bucket']
NUMERICAL_FEATURES = [
    'discount',
    'age',
    'tenure_months',
    'income_index',
    'web_visits_30d',
    'unit_margin_if_buy',
]

etp_framing = {
    'experiencia': (
        'Campaña outbound para clientes del segmento de retail financiero en la que cada fila del dataset '
        'representa un contacto individual previo al envío de una oferta de descuento.'
    ),
    'tarea': (
        'Predecir si el cliente concretará la compra del producto ofrecido (variable purchase) a partir de '
        'la información disponible antes de realizar la oferta.'
    ),
    'performance': (
        'Evaluar la calidad de las predicciones probabilísticas minimizando log-loss y maximizando métricas '
        'discriminativas como AUC, para tomar decisiones comerciales basadas en umbrales.'
    ),
    'framing': (
        'Se modelará como un problema de clasificación binaria con salida probabilística sobre purchase. '
        'Esto permite fijar umbrales de contacto para priorizar a los clientes con mayor propensión.'
    ),
    'politica_umbral': (
        'La operación aplicará un umbral sobre la probabilidad estimada: se contacta al cliente solo si la '
        'probabilidad supera t, lo que permite gestionar la capacidad y maximizar retorno esperado.'
    ),
}

for clave, descripcion in etp_framing.items():
    print(clave.upper())
    print(textwrap.fill(descripcion, width=100))
    print()


In [ ]:
# Punto 2: Métricas, pérdida y justificación
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), CATEGORICAL_FEATURES),
        ('numeric', Pipeline([('scaler', StandardScaler())]), NUMERICAL_FEATURES),
    ]
)

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(random_state=GLOBAL_SEED, max_iter=1000)),
    ]
)

X = df[FEATURE_COLUMNS]
y = df[TARGET_COLUMN]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=GLOBAL_SEED
)

model.fit(X_train, y_train)
y_proba = model.predict_proba(X_test)[:, 1]

metricas = {
    'log_loss': log_loss(y_test, y_proba),
    'auc': roc_auc_score(y_test, y_proba),
    'brier_score': brier_score_loss(y_test, y_proba),
}

for nombre, valor in metricas.items():
    print(f"{nombre}: {valor:.4f}")

mse_explicacion = (
    'La pérdida log-loss coincide con la maximización de la verosimilitud de un modelo Bernoulli y penaliza con mayor '
    'fuerza las probabilidades mal calibradas. Usar MSE como objetivo para clasificación binaria rompe esta relación '
    'probabilística, produce gradientes pobres cerca de los extremos y no prioriza la calibración, por lo que no es '
    'adecuado como criterio principal.'
)

print('
Justificación sobre el rechazo de MSE:')
print(textwrap.fill(mse_explicacion, width=100))
